In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Borůvka's Algorithm


## Towards parallel MST

- We know that each light edge must be in the MST.

- Prim and Kruskal pick light edges one at a time.

- Can we pick more than one at a time?



Consider a trivial cut: one vertex in one partition, everything else in the other.

We know that lowest weight edge from a vertex must be in MST. Call these the **vertex-bridges** of the graph.

<center>
<img src="figures/bridges.jpg" width=50%/>
</center>

Are we done? 

We haven't necessarily selected $n-1$ edges, which we need to have a MST.

The problem is some edges are selected by multiple vertices -- e.g., $a$ and $b$ both pick edge $(a,b)$.

So, we need to repeat this somehow, efficiently.

If we could collapse together the vertices connected by the selected edges, then we could solve a smaller version of this problem.

This is exactly what contraction is for!

<center>
    <img src="figures/borukva1.jpg" width=40%/>
    <img src="figures/borukva2.jpg" width=10%/>    
</center>

<br><br>

Due to light edge property, we know we should select $(e,f)$ which has minimum weight of $4$.

<br>

Notice that by collapsing vertices, we ignore internal edges -- e.g., if there were an edge from $c$ to $f$, we would ignore it when collapsing $c,d,f$. Why is this okay? 

## Borůvka's Algorithm

While there are edges remaining:

- select the minimum weight edge out of each vertex and contract each part defined by these edges into a vertex;

- remove self edges, and when there are redundant edges keep the minimum weight edge; and

- add all selected edges to the MST.

<br>

We need to:
- figure out the number of iterations
- figure out how to do the contraction

How many vertices will we contract at each iteration? Consider the example again:

<center>
    <img src="figures/borukva1.jpg" width=40%/>
</center>

If a partition has $k$ edges, how many vertices will be removed?

$k$

So, if the graph has $n$ vertices, and we pick one edge per vertex, what is the best and worst case on the number of vertices we can remove at each round?

**Best-case**: contract away $n-1$ vertices. We found the MST in one iteration!


<center>
    <img src="figures/borukva3.jpg" width=40%/>
</center>

**Worst-case**: contract away $\frac{n}{2}$ vertices. Each edge removes a single vertex.
<center>
    <img src="figures/borukva4.png" width=40%/>
</center>

<br>

So, we're guaranteed to remove $n/2$ vertices at each iteration.

Total number of contraction iterations is $O(\lg n)$

## Contracting in Borůvka's Algorithm

How can we contract these partitions?

<center>
    <img src="figures/borukva1.jpg" width=40%/>
</center>

In the above example, each partition happens to be a star graph, but it won't always be so:

<center>
    <img src="figures/borukva3.jpg" width=40%/>
</center>


<br>

Instead, we can apply star partitioning directly to the subgraph created by the vertex-bridges.


## Star partitioning in Borůvka

The only difference is that we only contract edges that are vertex bridges:

<center>
    <img src="figures/borukva5.jpg" width=40%/>
</center>

<center>
    <img src="figures/borukva6.png" width=20%/>
</center>

<br>

E.g., we don't contract $(b,d)$ because it is not a vertex bridge.

<br>

<p><span class="math display">\[\begin{array}{ll}  
1  ~~ \mathit{bridgeStarPartition}~(G=(V,E), i) =  
\\  
2 ~~  ~~~~\texttt{let}  
\\  
3  ~~ ~~~~~~~~\mathit{Eb} = \mathit{vertexBridges}~(G)  
\\  
4  ~~  ~~~~~~~~\mathit{P} = \left\{ u \mapsto (v,w) \in \mathit{Eb} \;|\; (  \mathit{flips}~(u) = T) \land (\mathit{flips}~(v) = H) \right\}  
\\  
5  ~~  ~~~~~~~~V' = V \setminus \mathit{domain}(P)  
\\  
6   ~~ ~~~~\texttt{in}  
\\   
7   ~~  ~~~~~~~~(V',P)  
\\   
8   ~~  ~~~~\texttt{end}  
\end{array}\]</span></p>


<br><br>

As in the original star contraction lecture, the expected number of vertices removed at each step is $\frac{n}{4}$. This is is not as good as $\frac{n}{2}$, but still good enough to ensure a logarithmic number of contraction iterations.






## Implementing Borůvka's Algorithm

While there are edges remaining:

- select the minimum weight edge out of each vertex and contract each part defined by these edges into a vertex;
  - Can implement with a min `reduce` at each vertex: span $\Rightarrow O(\lg |V|)$.
  
  
- remove self edges, and when there are redundant edges keep the minimum weight edge;
  - One run of star contraction: span $\Rightarrow O(\lg |V|)$.


- add all selected edges to the MST.
  - `filter`: span $\Rightarrow O(\lg |E|) \in O(\lg |V|)$


<br>

$S(|V|) = S(\frac{3|V|}{4}) + \lg |V| \in O(\lg^2 |V|)$